# Step 3A: Model Scoring evaluation

Using the results data set constructed in the `./notebooks/3a_model_scoring` Jupyter notebook, this notebook loads the data scores the observations. 

**Note:** This notebook will take about 1 minutes to execute all cells, depending on the compute configuration you have setup.

In [2]:
# import the libraries
import time

# For some data handling
import numpy as np
from pyspark.ml import PipelineModel
# for creating pipelines and model
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer

# Time the notebook execution. 
# This will only make sense if you "Run all cells"
tic = time.time()

# We will store and read each of these data sets in blob storage in an 
# Azure Storage Container on your Azure subscription.
# See https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md
# for details.

# The scoring uses the same feature engineering script used to train the model

results_table = 'results_output'

In [3]:
dbutils.widgets.removeAll()
dbutils.widgets.text("results_data", results_table)

In [4]:

# make predictions. The Pipeline does all the same operations on the test data
predictions =  spark.sql("SELECT * FROM " + dbutils.widgets.get("results_data"))

# Create the confusion matrix for the multiclass prediction results
# This result assumes a decision boundary of p = 0.5
conf_table = predictions.stat.crosstab('indexedLabel', 'prediction')
confuse = conf_table.toPandas()
confuse.head()

Out[ 3 ]: 
 indexedLabel_prediction 0.0 1.0 2.0 3.0 4.0
0 0.0 5785 28 53 24 2
1 1.0 9 11 0 0 0
2 2.0 10 0 12 0 0
3 3.0 8 1 0 31 0
4 4.0 18 1 0 0 12

The confusion matrix lists each true component failure in rows and the predicted value in columns. Labels numbered 0.0 corresponds to no component failures. Labels numbered 1.0 through 4.0 correspond to failures in one of the four components in the machine. As an example, the third number in the top row indicates how many days we predicted component 2 would fail, when no components actually did fail. The second number in the second row, indicates how many days we correctly predicted a component 1 failure within the next 7 days.

We read the confusion matrix numbers along the diagonal as correctly classifying the component failures. Numbers above the diagonal indicate the model incorrectly predicting a failure when non occured, and those below indicate incorrectly predicting a non-failure for the row indicated component failure.

When evaluating classification models, it is convenient to reduce the results in the confusion matrix into a single performance statistic. However, depending on the problem space, it is impossible to always use the same statistic in this evaluation. Below, we calculate four such statistics.

- **Accuracy**: reports how often we correctly predicted the labeled data. Unfortunatly, when there is a class imbalance (a large number of one of the labels relative to others), this measure is biased towards the largest class. In this case non-failure days.

Because of the class imbalance inherint in predictive maintenance problems, it is better to look at the remaining statistics instead. Here positive predictions indicate a failure.

- **Precision**: Precision is a measure of how well the model classifies the truely positive samples. Precision depends on falsely classifying negative days as positive.

- **Recall**: Recall is a measure of how well the model can find the positive samples. Recall depends on falsely classifying positive days as negative.

- **F1**: F1 considers both the precision and the recall. F1 score is the harmonic average of precision and recall. An F1 score reaches its best value at 1 (perfect precision and recall) and worst at 0.

These metrics make the most sense for binary classifiers, though they are still useful for comparision in our multiclass setting. Below we calculate these evaluation statistics for the selected classifier, and post them back to the AML workbench run time page for tracking between experiments.

In [6]:
# select (prediction, true label) and compute test error
# select (prediction, true label) and compute test error
# True positives - diagonal failure terms 
tp = confuse['1.0'][1]+confuse['2.0'][2]+confuse['3.0'][3]+confuse['4.0'][4]

# False positves - All failure terms - True positives
fp = np.sum(np.sum(confuse[['1.0', '2.0','3.0','4.0']])) - tp

# True negatives 
tn = confuse['0.0'][0]

# False negatives total of non-failure column - TN
fn = np.sum(np.sum(confuse[['0.0']])) - tn

# Accuracy is diagonal/total 
acc_n = tn + tp
acc_d = np.sum(np.sum(confuse[['0.0','1.0', '2.0','3.0','4.0']]))
acc = acc_n/acc_d

# Calculate precision and recall.
prec = tp/(tp+fp)
rec = tp/(tp+fn)

# Print the evaluation metrics to the notebook
print("Accuracy = %g" % acc)
print("Precision = %g" % prec)
print("Recall = %g" % rec )
print("F1 = %g" % (2.0 * prec * rec/(prec + rec)))
print("")

Accuracy = 0.974355
Precision = 0.377143
Recall = 0.594595
F1 = 0.461538

Remember that this is a simulated data set. We would expect a model built on real world data to behave very differently. The accuracy may still be close to one, but the precision and recall numbers would be much lower.

In [8]:
predictions.toPandas().head(20)

Out[ 5 ]: 
 machineID dt_truncated label_e \
0 27 2016-01-01 12:00:00 0.0 
1 27 2016-01-01 00:00:00 0.0 
2 27 2015-12-31 12:00:00 0.0 
3 27 2015-12-31 00:00:00 0.0 
4 27 2015-12-30 12:00:00 0.0 
5 27 2015-12-30 00:00:00 0.0 
6 63 2016-01-01 12:00:00 0.0 
7 63 2016-01-01 00:00:00 0.0 
8 63 2015-12-31 12:00:00 0.0 
9 63 2015-12-31 00:00:00 0.0 
10 63 2015-12-30 12:00:00 0.0 
11 63 2015-12-30 00:00:00 0.0 
12 158 2016-01-01 12:00:00 0.0 
13 158 2016-01-01 00:00:00 0.0 
14 158 2015-12-31 12:00:00 0.0 
15 158 2015-12-31 00:00:00 0.0 
16 158 2015-12-30 12:00:00 0.0 
17 158 2015-12-30 00:00:00 0.0 
18 389 2016-01-01 12:00:00 0.0 
19 389 2016-01-01 00:00:00 0.0 

 features indexedLabel \
0 (162.374561325, 445.713043834, 103.468531996, ... 0.0 
1 [169.63423645, 448.824823839, 100.134285273, 4... 0.0 
2 [168.028923157, 462.552245357, 97.5496715183, ... 0.0 
3 [162.926819659, 446.38100527, 99.8598322727, 3... 0.0 
4 [161.979750244, 445.396520667, 102.676947074, ... 0.0 
5 [167.273061702, 414.154289045, 97.0787477664, ... 0.0 
6 (170.665340293, 429.879779516, 102.52265497, 3... 0.0 
7 [170.909065392, 431.447522699, 101.58237207, 3... 0.0 
8 [168.68383159, 444.657960848, 100.900896359, 4... 0.0 
9 [172.052881411, 463.527143784, 96.3051763888, ... 0.0 
10 [169.785873523, 431.41255335, 99.9372850746, 3... 0.0 
11 [169.744079356, 465.405198219, 108.317072627, ... 0.0 
12 (171.493828712, 382.369420732, 99.0673772021, ... 0.0 
13 [168.186955023, 373.164849155, 100.27292124, 3... 0.0 
14 [171.302368793, 384.468098503, 101.990866038, ... 0.0 
15 [172.950717619, 439.978358371, 101.819695031, ... 0.0 
16 [164.436920443, 421.288136157, 103.451645545, ... 0.0 
17 [147.077021191, 452.425483615, 103.650360209, ... 0.0 
18 (174.812980875, 373.152557113, 100.378537101, ... 0.0 
19 [174.281654656, 370.171992292, 102.049819283, ... 0.0 

 indexedFeatures \
0 (162.374561325, 445.713043834, 103.468531996, ... 
1 [169.63423645, 448.824823839, 100.134285273, 4... 
2 [168.028923157, 462.552245357, 97.5496715183, ... 
3 [162.926819659, 446.38100527, 99.8598322727, 3... 
4 [161.979750244, 445.396520667, 102.676947074, ... 
5 [167.273061702, 414.154289045, 97.0787477664, ... 
6 (170.665340293, 429.879779516, 102.52265497, 3... 
7 [170.909065392, 431.447522699, 101.58237207, 3... 
8 [168.68383159, 444.657960848, 100.900896359, 4... 
9 [172.052881411, 463.527143784, 96.3051763888, ... 
10 [169.785873523, 431.41255335, 99.9372850746, 3... 
11 [169.744079356, 465.405198219, 108.317072627, ... 
12 (171.493828712, 382.369420732, 99.0673772021, ... 
13 [168.186955023, 373.164849155, 100.27292124, 3... 
14 [171.302368793, 384.468098503, 101.990866038, ... 
15 [172.950717619, 439.978358371, 101.819695031, ... 
16 [164.436920443, 421.288136157, 103.451645545, ... 
17 [147.077021191, 452.425483615, 103.650360209, ... 
18 (174.812980875, 373.152557113, 100.378537101, ... 
19 [174.281654656, 370.171992292, 102.049819283, ... 

 rawPrediction \
0 [194.818080952, 2.38343852109, 1.61969395882, ... 
1 [195.497549708, 1.9667072481, 1.4855159663, 0.... 
2 [195.302874731, 2.15297367665, 1.45020654836, ... 
3 [195.395743401, 2.11349766949, 1.46228862084, ... 
4 [194.984577984, 2.41947518418, 1.54131393302, ... 
5 [164.346557514, 29.7588608332, 3.5714182125, 1... 
6 [190.827962879, 3.09262503493, 2.58638885194, ... 
7 [192.828662759, 2.17306795386, 1.50501273278, ... 
8 [192.903016667, 2.02545719314, 1.50304922475, ... 
9 [192.728249649, 2.03981076619, 1.55680835096, ... 
10 [192.965996174, 2.07607150943, 1.45782071013, ... 
11 [174.651251129, 0.698327609769, 1.73962681977,... 
12 [155.108324675, 38.7443723342, 2.48626059478, ... 
13 [124.652803419, 73.0339095098, 1.10925205227, ... 
14 [143.650334865, 53.3580536805, 0.785911344795,... 
15 [193.74792578, 2.63516315764, 1.50327102927, 1... 
16 [162.74146305, 32.3064496173, 2.51446443785, 1... 
17 [190.422246841, 2.06424539704, 1.16495149178, ... 
18 [153.318897025, 40.6819908401, 3.80577071264, ... 
19 [110.903867526, 83.5105953164, 1.9

In [9]:
print(predictions.summary())

DataFrame[summary: string, machineID: string, label_e: string, indexedLabel: string, prediction: string]

In [10]:
predictions.explain()

== Physical Plan ==
*(1) FileScan parquet default.results_output[machineID#2L,dt_truncated#3,label_e#4,features#5,indexedLabel#6,indexedFeatures#7,rawPrediction#8,probability#9,prediction#10] Batched: true, Format: Parquet, Location: InMemoryFileIndex[dbfs:/user/hive/warehouse/results_output], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<machineID:bigint,dt_truncated:timestamp,label_e:double,features:struct<type:tinyint,size:i...

# Conclusion

This concludes this scenario. You can modify these notebooks to customize your own use case solution.